In [1]:
import math

## Delhi

In [ ]:
import googlemaps
import pandas as pd
import numpy as np
from datetime import datetime

# --- Configuration ---
API_KEY = "put your api key "

# Define the area and grid size as before
AREA_BOUNDS = {
    "sw": (28.341944, 76.811111),  # South-West corner (lat, lon)
    "ne": (28.928889, 77.473056)   # North-East corner
}
GRID_SIZE_X = 7
GRID_SIZE_Y = 7
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_DELHI{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Main Script ---
gmaps = googlemaps.Client(key=API_KEY)

def generate_grid_points(bounds, num_x, num_y):
    lats = np.linspace(bounds["sw"][0], bounds["ne"][0], num_y)
    lons = np.linspace(bounds["sw"][1], bounds["ne"][1], num_x)
    return [(lat, lon) for lat in lats for lon in lons]

grid_points = generate_grid_points(AREA_BOUNDS, GRID_SIZE_X, GRID_SIZE_Y)

all_path_points = []
total_paths = (GRID_SIZE_X * GRID_SIZE_Y) ** 2 - (GRID_SIZE_X * GRID_SIZE_Y)
path_counter = 0


for i, origin in enumerate(grid_points):
    for j, destination in enumerate(grid_points):
        if i == j:
            continue
        
        path_counter += 1
        print(f"  Processing path {path_counter}/{total_paths} (Point {i} -> Point {j})")

     
        # Request directions for this specific pair
        directions_result = gmaps.directions(origin,
                                                destination,
                                                mode="driving",
                                                departure_time=datetime.now())

        if not directions_result:
            continue

        # --- Extract Key Information ---
        route = directions_result[0] # Use the primary recommended route
        leg = route['legs'][0]
        
        # Calculate congestion factor for this specific path
        duration_val = leg['duration']['value']
        duration_traffic_val = leg.get('duration_in_traffic', {}).get('value', duration_val)
        congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

        # --- Decode the Polyline to get actual road coordinates ---
        encoded_polyline = route['overview_polyline']['points']
        decoded_path = googlemaps.convert.decode_polyline(encoded_polyline)

        # Create a unique ID for this path
        path_id = f"Point {i} -> Point {j}"
        
        # For each coordinate in the decoded path, create a row for Tableau
        for order, point in enumerate(decoded_path):
            all_path_points.append({
                'Path_ID': path_id,
                'Point_Order': order + 1,
                'Latitude': point['lat'],
                'Longitude': point['lng'],
                'Congestion_Factor': round(congestion_factor, 2),
                'Distance_Text': leg['distance']['text'],
                'Duration_in_Traffic_Text': leg.get('duration_in_traffic', {}).get('text', 'N/A')
            })




# Convert the final list into a DataFrame
tableau_df = pd.DataFrame(all_path_points)

# Save the detailed path data to a new CSV file
tableau_df.to_csv(OUTPUT_FILENAME, index=False)

print(f"\n Successfully generated detailed path data!")
print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")

  Processing path 1/2352 (Point 0 -> Point 1)
  Processing path 2/2352 (Point 0 -> Point 2)
  Processing path 3/2352 (Point 0 -> Point 3)
  Processing path 4/2352 (Point 0 -> Point 4)
  Processing path 5/2352 (Point 0 -> Point 5)
  Processing path 6/2352 (Point 0 -> Point 6)
  Processing path 7/2352 (Point 0 -> Point 7)
  Processing path 8/2352 (Point 0 -> Point 8)
  Processing path 9/2352 (Point 0 -> Point 9)
  Processing path 10/2352 (Point 0 -> Point 10)
  Processing path 11/2352 (Point 0 -> Point 11)
  Processing path 12/2352 (Point 0 -> Point 12)
  Processing path 13/2352 (Point 0 -> Point 13)
  Processing path 14/2352 (Point 0 -> Point 14)
  Processing path 15/2352 (Point 0 -> Point 15)
  Processing path 16/2352 (Point 0 -> Point 16)
  Processing path 17/2352 (Point 0 -> Point 17)
  Processing path 18/2352 (Point 0 -> Point 18)
  Processing path 19/2352 (Point 0 -> Point 19)
  Processing path 20/2352 (Point 0 -> Point 20)
  Processing path 21/2352 (Point 0 -> Point 21)
  Processi

## bangalore 


In [ ]:
import googlemaps
import pandas as pd
import numpy as np
from datetime import datetime

# --- Configuration ---
# Replace this placeholder with your full, valid Google Maps API key.
API_KEY = "put your api key "

# MODIFICATION: Define the area using the specific coordinates you provided for Bangalore.
AREA_BOUNDS = {
    "sw": (12.838889, 77.396667),  # South-West corner (lat, lon)
    "ne": (13.107778, 77.742500)   # North-East corner
}
GRID_SIZE_X = 7
GRID_SIZE_Y = 7

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_Bangalore_Grid_{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Main Script ---
gmaps = googlemaps.Client(key=API_KEY)

def generate_grid_points(bounds, num_x, num_y):
    """Generates a grid of lat/lon points within the defined bounds."""
    lats = np.linspace(bounds["sw"][0], bounds["ne"][0], num_y)
    lons = np.linspace(bounds["sw"][1], bounds["ne"][1], num_x)
    return [(lat, lon) for lat in lats for lon in lons]

grid_points = generate_grid_points(AREA_BOUNDS, GRID_SIZE_X, GRID_SIZE_Y)

all_path_points = []
total_paths = (GRID_SIZE_X * GRID_SIZE_Y) ** 2 - (GRID_SIZE_X * GRID_SIZE_Y)
path_counter = 0

print(f"Starting traffic analysis for Bangalore (Grid Method)...")
print("-" * 50)

for i, origin in enumerate(grid_points):
    for j, destination in enumerate(grid_points):
        if i == j:
            continue
        
        path_counter += 1
        print(f"  Processing path {path_counter}/{total_paths} (Point {i} -> Point {j})")

        try:
            # Request directions for this specific pair
            directions_result = gmaps.directions(origin,
                                                 destination,
                                                 mode="driving",
                                                 departure_time=datetime.now())

            if not directions_result:
                continue

            # --- Extract Key Information ---
            route = directions_result[0] # Use the primary recommended route
            leg = route['legs'][0]
            
            # Calculate congestion factor for this specific path
            duration_val = leg['duration']['value']
            duration_traffic_val = leg.get('duration_in_traffic', {}).get('value', duration_val)
            congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

            # --- Decode the Polyline to get actual road coordinates ---
            encoded_polyline = route['overview_polyline']['points']
            decoded_path = googlemaps.convert.decode_polyline(encoded_polyline)

            # Create a unique ID for this path
            path_id = f"Point {i} -> Point {j}"
            
            # For each coordinate in the decoded path, create a row for Tableau
            for order, point in enumerate(decoded_path):
                all_path_points.append({
                    'Path_ID': path_id,
                    'Point_Order': order + 1,
                    'Latitude': point['lat'],
                    'Longitude': point['lng'],
                    'Congestion_Factor': round(congestion_factor, 2),
                    'Distance_Text': leg['distance']['text'],
                    'Duration_in_Traffic_Text': leg.get('duration_in_traffic', {}).get('text', 'N/A')
                })
        except Exception as e:
            print(f"    An error occurred for path {i}->{j}: {e}")
            continue

# Convert the final list into a DataFrame
if all_path_points:
    tableau_df = pd.DataFrame(all_path_points)
    # Save the detailed path data to a new CSV file
    tableau_df.to_csv(OUTPUT_FILENAME, index=False)
    print(f"\n Successfully generated detailed path data!")
    print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")
else:
    print("\nNo data was generated. The output file was not created.")


Starting traffic analysis for Bangalore (Grid Method)...
--------------------------------------------------
  Processing path 1/2352 (Point 0 -> Point 1)
  Processing path 2/2352 (Point 0 -> Point 2)
  Processing path 3/2352 (Point 0 -> Point 3)
  Processing path 4/2352 (Point 0 -> Point 4)
  Processing path 5/2352 (Point 0 -> Point 5)
  Processing path 6/2352 (Point 0 -> Point 6)
  Processing path 7/2352 (Point 0 -> Point 7)
  Processing path 8/2352 (Point 0 -> Point 8)
  Processing path 9/2352 (Point 0 -> Point 9)
  Processing path 10/2352 (Point 0 -> Point 10)
  Processing path 11/2352 (Point 0 -> Point 11)
  Processing path 12/2352 (Point 0 -> Point 12)
  Processing path 13/2352 (Point 0 -> Point 13)
  Processing path 14/2352 (Point 0 -> Point 14)
  Processing path 15/2352 (Point 0 -> Point 15)
  Processing path 16/2352 (Point 0 -> Point 16)
  Processing path 17/2352 (Point 0 -> Point 17)
  Processing path 18/2352 (Point 0 -> Point 18)
  Processing path 19/2352 (Point 0 -> Point 19

## Mumbai

In [ ]:
import googlemaps
import pandas as pd
import numpy as np
from datetime import datetime

# --- Configuration ---
# Replace this placeholder with your full, valid Google Maps API key.
API_KEY = "put your api key "

# MODIFICATION: Define the area using the specific coordinates you provided for Mumbai.
AREA_BOUNDS = {
    "sw": (18.905278, 72.765278),  # South-West corner (lat, lon)
    "ne": (19.320833, 73.113056)   # North-East corner
}
GRID_SIZE_X = 7
GRID_SIZE_Y = 7

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_Mumbai_Grid_{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Main Script ---
gmaps = googlemaps.Client(key=API_KEY)

def generate_grid_points(bounds, num_x, num_y):
    """Generates a grid of lat/lon points within the defined bounds."""
    lats = np.linspace(bounds["sw"][0], bounds["ne"][0], num_y)
    lons = np.linspace(bounds["sw"][1], bounds["ne"][1], num_x)
    return [(lat, lon) for lat in lats for lon in lons]

grid_points = generate_grid_points(AREA_BOUNDS, GRID_SIZE_X, GRID_SIZE_Y)

all_path_points = []
total_paths = (GRID_SIZE_X * GRID_SIZE_Y) ** 2 - (GRID_SIZE_X * GRID_SIZE_Y)
path_counter = 0

print(f"Starting traffic analysis for Mumbai (Grid Method)...")
print("-" * 50)

for i, origin in enumerate(grid_points):
    for j, destination in enumerate(grid_points):
        if i == j:
            continue
        
        path_counter += 1
        print(f"  Processing path {path_counter}/{total_paths} (Point {i} -> Point {j})")

        try:
            # Request directions for this specific pair
            directions_result = gmaps.directions(origin,
                                                 destination,
                                                 mode="driving",
                                                 departure_time=datetime.now())

            if not directions_result:
                continue

            # --- Extract Key Information ---
            route = directions_result[0] # Use the primary recommended route
            leg = route['legs'][0]
            
            # Calculate congestion factor for this specific path
            duration_val = leg['duration']['value']
            duration_traffic_val = leg.get('duration_in_traffic', {}).get('value', duration_val)
            congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

            # --- Decode the Polyline to get actual road coordinates ---
            encoded_polyline = route['overview_polyline']['points']
            decoded_path = googlemaps.convert.decode_polyline(encoded_polyline)

            # Create a unique ID for this path
            path_id = f"Point {i} -> Point {j}"
            
            # For each coordinate in the decoded path, create a row for Tableau
            for order, point in enumerate(decoded_path):
                all_path_points.append({
                    'Path_ID': path_id,
                    'Point_Order': order + 1,
                    'Latitude': point['lat'],
                    'Longitude': point['lng'],
                    'Congestion_Factor': round(congestion_factor, 2),
                    'Distance_Text': leg['distance']['text'],
                    'Duration_in_Traffic_Text': leg.get('duration_in_traffic', {}).get('text', 'N/A')
                })
        except Exception as e:
            print(f"    An error occurred for path {i}->{j}: {e}")
            continue

# Convert the final list into a DataFrame
if all_path_points:
    tableau_df = pd.DataFrame(all_path_points)
    # Save the detailed path data to a new CSV file
    tableau_df.to_csv(OUTPUT_FILENAME, index=False)
    print(f"\n Successfully generated detailed path data!")
    print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")
else:
    print("\nNo data was generated. The output file was not created.")


## Kolkata:

In [ ]:
import googlemaps
import pandas as pd
import numpy as np
from datetime import datetime

# --- Configuration ---
# Replace this placeholder with your full, valid Google Maps API key.
API_KEY = "put your api key "

# MODIFICATION: Define the area using the specific coordinates you provided for Kolkata.
AREA_BOUNDS = {
    "sw": (22.114167, 87.948333),  # South-West corner (lat, lon)
    "ne": (22.955833, 88.653889)   # North-East corner
}
GRID_SIZE_X = 7
GRID_SIZE_Y = 7

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_Kolkata_Grid_{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Main Script ---
gmaps = googlemaps.Client(key=API_KEY)

def generate_grid_points(bounds, num_x, num_y):
    """Generates a grid of lat/lon points within the defined bounds."""
    lats = np.linspace(bounds["sw"][0], bounds["ne"][0], num_y)
    lons = np.linspace(bounds["sw"][1], bounds["ne"][1], num_x)
    return [(lat, lon) for lat in lats for lon in lons]

grid_points = generate_grid_points(AREA_BOUNDS, GRID_SIZE_X, GRID_SIZE_Y)

all_path_points = []
total_paths = (GRID_SIZE_X * GRID_SIZE_Y) ** 2 - (GRID_SIZE_X * GRID_SIZE_Y)
path_counter = 0

print(f"Starting traffic analysis for Kolkata (Grid Method)...")
print("-" * 50)

for i, origin in enumerate(grid_points):
    for j, destination in enumerate(grid_points):
        if i == j:
            continue
        
        path_counter += 1
        print(f"  Processing path {path_counter}/{total_paths} (Point {i} -> Point {j})")

        try:
            # Request directions for this specific pair
            directions_result = gmaps.directions(origin,
                                                 destination,
                                                 mode="driving",
                                                 departure_time=datetime.now())

            if not directions_result:
                continue

            # --- Extract Key Information ---
            route = directions_result[0] # Use the primary recommended route
            leg = route['legs'][0]
            
            # Calculate congestion factor for this specific path
            duration_val = leg['duration']['value']
            duration_traffic_val = leg.get('duration_in_traffic', {}).get('value', duration_val)
            congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

            # --- Decode the Polyline to get actual road coordinates ---
            encoded_polyline = route['overview_polyline']['points']
            decoded_path = googlemaps.convert.decode_polyline(encoded_polyline)

            # Create a unique ID for this path
            path_id = f"Point {i} -> Point {j}"
            
            # For each coordinate in the decoded path, create a row for Tableau
            for order, point in enumerate(decoded_path):
                all_path_points.append({
                    'Path_ID': path_id,
                    'Point_Order': order + 1,
                    'Latitude': point['lat'],
                    'Longitude': point['lng'],
                    'Congestion_Factor': round(congestion_factor, 2),
                    'Distance_Text': leg['distance']['text'],
                    'Duration_in_Traffic_Text': leg.get('duration_in_traffic', {}).get('text', 'N/A')
                })
        except Exception as e:
            print(f"    An error occurred for path {i}->{j}: {e}")
            continue

# Convert the final list into a DataFrame
if all_path_points:
    tableau_df = pd.DataFrame(all_path_points)
    # Save the detailed path data to a new CSV file
    tableau_df.to_csv(OUTPUT_FILENAME, index=False)
    print(f"\n Successfully generated detailed path data!")
    print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")
else:
    print("\nNo data was generated. The output file was not created.")
